<a href="https://colab.research.google.com/github/soujanya-vattikolla/MongoDB-for-Python-Developers-/blob/main/Chapter2%20UserFacingBackend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cursor Methods and Aggregation Equivalents**

* The find method is always going to return a cursor to us.
* Limit method: we can limit the number of documents that return in the cursor using .limit method.

* **Aggregation framework**
* In this we use $limit.

* **Sort** takes two parameters - the key that we're sorting on and the sorting order.

* **Aggregation framework** 

* In sort method, a dictionary with the field we want to sort on and the sorting order.

* **Skip**
* A skip method allows us to skip documents in a collection. So only documents we did not skip will appear in the cursor.

In [ ]:
.limit() == $limit
.sort() == $sort 
.skip() == $skip

Question:

Which of the following aggregation stages have equivalent cursor methods?

$sort

Sorting can be accomplished with the .sort() cursor method.

$skip

Skipping can be accomplished with the .skip() cursor method.

$limit

Limiting can be accomplished with the .limit() cursor method.

**Problem**:<br>

User Story <br>

"As a user, I'd like to get the next page of results for my query by scrolling down in the main window of the application." <br>

Task <br>

Modify the method get_movies in db.py to allow for paging. You can see how the page is parsed and sent in the api_search_movies method from movies.py.<br>

In [ ]:
def get_movies(filters, page, movies_per_page):
    query, sort, project = build_query_sort_project(filters)
    if project and sort:
        cursor = db.movies.find(query, project).sort(sort)
    elif project:
        cursor = db.movies.find(query).sort(sort)
    else:
        cursor = db.movies.find(query)

    total_num_movies = cursor.count()

    # here's an implementation of paging using skip() and limit()
    movies = cursor.skip(movies_per_page * page).limit(movies_per_page)

    return (list(movies), total_num_movies)

**Basic Aggregation**

* Aggregation is a pipeline
    *  Pipelines are composed of stages, broad units of work.
    *  Within stages, expressions are used to specify individual units of work.
* Expressions are functions.

In [ ]:
# Aggregation

{ "$add": ["$a", "$b"]  }

**Problem:**<br>

User Story<br>

"As a user, I want to be able to filter cast search results by one facet, metacritic rating."<br>

Task <br>

For this Ticket, you'll be required to implement one method in db.py, get_movies_faceted, so the MFlix application can perform faceted searches.



In [ ]:
def get_movies_faceted(filters, page, movies_per_page):
    sort_key = "tomatoes.viewer.numReviews"

    pipeline = []

    if "cast" in filters:
        pipeline.extend([{
            "$match": {"cast": {"$in": filters.get("cast")}}
        }, {
            "$sort": {sort_key: DESCENDING}
        }])
    else:
        raise AssertionError("No filters to pass to faceted search!")

    counting = pipeline[:]
    count_stage = { "$count": "count" }
    counting.append(count_stage)

    skip_stage = { "$skip": movies_per_page * page }
    limit_stage =  { "$limit": movies_per_page }
    facet_stage = {
        "$facet": {
            "runtime": [{
                "$bucket": {
                    "groupBy": "$runtime",
                    "boundaries": [0, 60, 90, 120, 180],
                    "default": "other",
                    "output": {
                        "count": {"$sum": 1}
                    }
                }
            }],
            "rating": [{
                "$bucket": {
                    "groupBy": "$metacritic",
                    "boundaries": [0, 50, 70, 90, 100],
                    "default": "other",
                    "output": {
                        "count": {"$sum": 1}
                    }
                }
            }],
            "movies": [{
                "$addFields": {
                    "title": "$title"
                }
            }]
        }
    }


    # here's where the stages are appended to the pipeline object
    pipeline.extend([skip_stage, limit_stage, facet_stage])


    try:
        movies = list(db.movies.aggregate(pipeline, allowDiskUse=True))[0]
        count = list(db.movies.aggregate(counting, allowDiskUse=True))[
            0].get("count")
        return (movies, count)
    except OperationFailure:
        raise OperationFailure(
            "Results too large to sort, be more restrictive in filter")

**Upserts vs. Updates**<br>

Sometimes, we want to update a document, but we're not sure if it exists in the collection.<br>

We can use an "upsert" to update a document if it exists, and insert it if it does not exist.<br>

In the following example, we're not sure if this video game exists in our collection, but we want to make sure there is a document in the collection that contains the correct data.<br>

This operation may do one of two things:<br>

If the predicate matches a document, update the document to contain the correct data.<br>

If the document doesn't exist, create the desired document.

**Problem:**<br>

Which of the following is true about InsertOneResult?<br>

* It contains the ``_id`` of an inserted document.

    * This can be accessed with the inserted_id property.

* It can tell us whether the operation was acknowledged by the server.

    * This can be accessed with the acknowledged property.

**Problem:**<br>

User Story<br>

"As a user, I should be able to register for an account, log in, and logout."<br>

Task<br>

For this Ticket, you'll be required to implement all the methods in db.py that are called by the API endpoints in user.py. Specifically, you'll implement:<br>

* get_user
* add_user
* login_user
* logout_user
* get_user_session
* delete_user <br>
For this ticket, you will need to use the find_one(), update_one() and delete_one() methods.

In [ ]:
def get_user(email):
    return db.users.find_one({"email": email})

def add_user(name, email, hashedpw):
    try:
        db.users.insert_one(
          {"name": name, "email": email, "password": hashedpw}
        )
        return {"success": True}
    except DuplicateKeyError:
        return {"error": "A user with the given email already exists."}

def login_user(email, jwt):
    try:
        db.sessions.update_one(
            {"user_id": email}, {"$set": {"jwt": jwt}}, upsert=True)
        return {"success": True}
    except Exception as e:
        return {"error": e}

def logout_user(email):
    try:
        db.sessions.delete_one({"user_id": email})
        return {"success": True}
    except Exception as e:
        return {"error": e}

def get_user_session(email):
    try:
        return db.sessions.find_one({"user_id": email})
    except Exception as e:
        return {"error": e}

def delete_user(email):
    try:
        db.users.delete_one({"email": email})
        db.sessions.delete_one({"user_id": email})
        if get_user(email) is None:
            return {"success": True}
        else:
            raise ValueError("Deletion unsuccessful")
    except Exception as e:
        return {"error": e}

**Write Concerns**

**writeConcern:{w:1}**<br>
* Only requests an acknowledgement that one node applied the write.
* This is the default writeConcern in MongoDB

* **w: majorit**y ensures that writes are committed by a majority of nodes.
    * Slower, but very durable

* **w: 0** does not ensure that a write was committed by any nodes.
    * Very fast, but less durable

**Problem:**<br>

Which of the following Write Concerns are valid in a 3-node replica set?<br>

* w: 0

    * This will not ask for an acknowledgement from any of the nodes in the set.

* w: 1

    * This will only ask for an acknowledgement from one of the nodes in the set.

* w: majority

    * This will ask for an acknowledgement from a majority of nodes in the set.

**Problem:**<br>

Task <br>

For this ticket, you'll be required to increase the durability of the add_user method from the default write concern of w: 1. <br>

When a new user registers for MFlix, their information must be added to the database before they can do anything else on the site. For this reason, we want to make sure that the data written by the add_user method will not be rolled back. <br>

We can completely eliminate the chances of a rollback by increasing the write durability of the add_user method. To use a non-default write concern with a database operation, use Pymongo's with_options flag when issuing the query.

In [ ]:
def add_user(name, email, hashedpw):
    try:
        # this is where a write_concern keyword argument is provided
        db.users.with_options(write_concern=WriteConcern(w="majority")) \
            .insert_one(
                {"name": name, "email": email, "password": hashedpw}
        )
        return {"success": True}
    except DuplicateKeyError:
        return {"error": "A user with the given email already exists."}

* Which of the following write concerns are more durable than the default?<br>

    * w: 2, w: "majority"

    * In a 3-node replica set, these two write concerns will both wait until 2 nodes have applied a write. This is because 2 out of 3 nodes is a majority, and waiting for 2 nodes to apply a write is more durable than only waiting for 1 node to apply it.

**Basic Updates**

**update operations**<br>

  * update_one
  * update_many

* Update operations return an UpdateResult
  * acknowledged, matched_count, modified_count, and upserted_id.
  * In the case of an upsert, modified_count and matched_count will be 0.

Which of the following are valid update operators in Pymongo?<br>

* $set

    * This will replace the value of a field with the specified value.

* $push

    * This will append a specified value to the end of an array field.

* $inc

    * This will increment a field by a specified amount.

**Ticket: User Preferences**<br>
**Problem:**<br>

User Story<br>

"As a user, I want to be able to store preferences such as my favorite cast member and preferred language."<br>

Task<br>

For this Ticket, you'll be required to implement one method in db.py, update_prefs. This method allows updates to be made to the "preferences" field in the users collection.

In [ ]:
def update_prefs(email, prefs):
    '''
    Updates user preferences
    '''
    prefs = {} if prefs is None else prefs
    try:
        response = db.users.update_one(
            {"email": email},
            {"$set": {"preferences": prefs}}
        )
        if response.matched_count == 0:
            return {'error': 'no user found'}
        else:
            return response
    except Exception as e:
        return {'error': str(e)}

**Basic Joins**

* Join two collections of data.
* Use new expressive $ lookup

**Problem:**<br>

Why did we use a let expression with expressive $lookup, when joining the comments and the movies collection?

* To use fields from the movies collection in the pipeline .

    * The only way we can use fields from the movies collection in the pipeline is by defining those variables in the let expression.

**Ticket: Get Comments**<br>
**Problem:**<br>

User Story<br>

"As a user, I want to be able to view comments for a movie when I look at the movie detail page."<br>

Task<br>

For this ticket, you'll be required to extend the get_movie method in db.py so that it also fetches the comments for a given movie.<br>

The comments should be returned in order from most recent to least recent using the date key.<br>

Movie comments are stored in the comments collection, so this task can be accomplished by performing a $lookup.<br>

In [ ]:
def get_movie(id):
    try:
        # here's the pipeline used to join comments
        pipeline = [
            {
                # find the current movie in the "movies" collection
                "$match": {
                    "_id": ObjectId(id)
                }
            },
            {
                "$lookup": {
                    "from": "comments",
                    "let": { "id": "$_id" },
                    "pipeline": [
                        # only join comments with matching movie_id
                        {
                            "$match": {
                                "$expr": {"$eq": ["$movie_id", "$$id"]}}
                        },
                        # sort comments in descending order by date
                        {
                            "$sort": {"date": -1}
                        }
                    ],
                    # call embedded field comments
                    "as": "comments"
                }
            }
        ]
        return db.movies.aggregate(pipeline).next()
    except (StopIteration) as _:
        return None

**Ticket: Create/Update Comments**<br>
Problem:<br>

User Story<br>

"As a user, I want to be able to post comments to a movie page as well as edit my own comments."<br>

Task<br>

For this ticket, you'll be required to implement two methods in db.py, add_comment and update_comment.<br>

Ensure that update_comment only allows users to update their own comments, and no one else's comments.

In [ ]:
def add_comment(movie_id, user, comment, date):
    comment_doc = {
        "name": user.name,
        "email": user.email,
        "movie_id": ObjectId(movie_id),
        "text": comment,
        "date": date
    }
    return db.comments.insert_one(comment_doc)

def update_comment(comment_id, user_email, text, date):
    response = db.comments.update_one(
        # we used the comment_id and user_email to verify that the user has
        # permission to edit this comment
        {"_id": ObjectId(comment_id), "email": user_email},
        {"$set": {"text": text, "date": date}}
    )
    return response

**Basic Deletes**

* **delete_one** will delete the first document that matches the supplied predicate
* **delete_many** will delete all documents matching the supplied predicate
* The number of documents deleted can be accessed via the **deleted_count** property on the DeleteResult object returned from a delete operation.

In [ ]:
Problem:

Which of the following is true about deleting documents in Pymongo?

* DeleteResult objects contain the number of deleted documents.

    * We can access this with the .acknowledged property of the DeleteResult object.

* delete_many() can delete any number of documents.

    * That's why it's called delete many, and not, say, delete just a couple of 'em.

* delete_one() can only delete one document.

    * This method can be helpful when our operation only intends to delete one document.

**Ticket: Delete Comments**<br>
Problem:<br>

User Story<br>

"As a user, I want to be able to delete my own comments."<br>

Task<br>

For this ticket, you'll be required to modify one method in db.py, delete_comment. Ensure the delete operation is limited so only the user can delete their own comments, but not anyone else's comments.

In [ ]:
def delete_comment(comment_id, user_email):
    response = db.comments.delete_one(
        {"_id": ObjectId(comment_id), "email": user_email}
    )
    return response